In [1]:
import pandas as pd
import glob
import os

In [2]:
data_folder = os.path.expanduser("~/DataS")

all_files = glob.glob(os.path.join(data_folder, "2016-*.csv"))

frames = []
for file in all_files:
    df = pd.read_csv(file, low_memory = False)
    frames.append(df)

data = pd.concat(frames, ignore_index = True)

# Checking...
data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.)
0,2016-07-31T23:56:00,2016-08-01T00:13:00,A04,Eiran Sairaala,A27,Mannerheimintie,1517,1001
1,2016-07-31T23:54:00,2016-08-01T00:08:00,B08,Sörnäisten metroasema,A35,Apollonkatu,2913,816
2,2016-07-31T23:54:00,2016-08-01T00:01:00,A23,Rautatientori / itä,A35,Apollonkatu,1564,442
3,2016-07-31T23:47:00,2016-07-31T23:59:00,C02,Ooppera,B04,Karhupuisto,1878,696
4,2016-07-31T23:46:00,2016-07-31T23:55:00,A17,Liisanpuistikko,A16,Merisotilaantori,1723,554


In [5]:
data["Departure"] = pd.to_datetime(data["Departure"], errors="coerce")
data["Return"] = pd.to_datetime(data["Return"], errors="coerce")

data["Departure_hour"] = data["Departure"].dt.hour
data["Return_hour"] = data["Return"].dt.hour

# Checking...
data.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.),Departure_hour,Return_hour
0,2016-07-31 23:56:00,2016-08-01 00:13:00,A04,Eiran Sairaala,A27,Mannerheimintie,1517,1001,23.0,0.0
1,2016-07-31 23:54:00,2016-08-01 00:08:00,B08,Sörnäisten metroasema,A35,Apollonkatu,2913,816,23.0,0.0
2,2016-07-31 23:54:00,2016-08-01 00:01:00,A23,Rautatientori / itä,A35,Apollonkatu,1564,442,23.0,0.0
3,2016-07-31 23:47:00,2016-07-31 23:59:00,C02,Ooppera,B04,Karhupuisto,1878,696,23.0,23.0
4,2016-07-31 23:46:00,2016-07-31 23:55:00,A17,Liisanpuistikko,A16,Merisotilaantori,1723,554,23.0,23.0


In [6]:
departures_by_hour = data.groupby(
    ["Departure station name", "Departure_hour"]
).size().reset_index(name="departures")

returns_by_hour = data.groupby(
    ["Return station name", "Return_hour"]
).size().reset_index(name="returns")

departures_by_hour.head(), returns_by_hour.head()

(  Departure station name  Departure_hour  departures
 0            Apollonkatu             0.0         107
 1            Apollonkatu             1.0          53
 2            Apollonkatu             2.0          23
 3            Apollonkatu             3.0          19
 4            Apollonkatu             4.0          12,
   Return station name  Return_hour  returns
 0         Apollonkatu          0.0      158
 1         Apollonkatu          1.0      105
 2         Apollonkatu          2.0       41
 3         Apollonkatu          3.0       48
 4         Apollonkatu          4.0       34)

In [8]:
departures_by_hour.rename(columns = {
    "Departure station name": "Station",
    "Departure_hour": "Hour"
}, inplace = True)

returns_by_hour.rename(columns = {
    "Return station name": "Station",
    "Return_hour": "Hour"
}, inplace = True)

hourly_summary = pd.merge(
    departures_by_hour, 
    returns_by_hour, 
    on = ["Station", "Hour"], 
    how = "outer"
).fillna(0)

hourly_summary

,Station,Hour,departures,returns
0,Apollonkatu,0.0,107.0,158.0
1,Apollonkatu,1.0,53.0,105.0
2,Apollonkatu,2.0,23.0,41.0
3,Apollonkatu,3.0,19.0,48.0
4,Apollonkatu,4.0,12.0,34.0
...,...,...,...,...
1182,Ympyrätalo,19.0,613.0,914.0
1183,Ympyrätalo,20.0,500.0,861.0
1184,Ympyrätalo,21.0,391.0,609.0
1185,Ympyrätalo,22.0,303.0,421.0


In [9]:
hourly_summary.to_csv("hourly_summary.csv", index = False)